In [3]:
## mashup script for batch changes
import math
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
from pyproj import proj
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [4]:
# variables
apiToken = "2ef04ab67ab414b7ac6c7815235ace6f9cdf3ab79b9e0874beb053a9dfa9bb682d7bbd1a3117b68e"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=1)
recordsPerPage = 5000

In [5]:
# target data
formIDTarget = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" #change this to the app you want to update
formdataTarget = fulcrum.forms.find(formIDTarget)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountTarget = fulcrum.forms.find(formIDTarget)['form']['record_count']
pagesTarget = math.ceil(recordCountTarget / recordsPerPage)

In [9]:
## get target data
if pagesTarget > 1:
    for p in range(1, pagesTarget + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': formIDTarget, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataTarget.extend(dataPage)
        else:
            dataTarget = dataPage
else:
    dataTarget = fulcrum.records.search(url_params={'form_id': formIDTarget, 'page': 1, 'per_page': recordsPerPage})['records']

In [10]:
noloc = missmatch = match = notfilter = incomplete = 0
rfi = nar = match = oh = incomplete = 0

In [12]:
for record in dataTarget:
    if record['status'] == 'Inspection required':
        record['status'] = 'Request of inspection (RFI) - طلب معاينة'
        rfi = rfi+1
        print('rfi',rfi)
        recordUpdated = fulcrum.records.update(record['id'], record)
    elif record['status'] == 'Action required':
        record['status']= 'Issue in place - قضية موجودة'
        nar = nar+1
        print('nar',nar)
        recordUpdated = fulcrum.records.update(record['id'], record)
    elif record['status'] == 'No action required':
        record['status'] = 'No action required - لا حاجة لأي إجراء'
        nar = nar+1
        print('nar',nar)
        recordUpdated = fulcrum.records.update(record['id'], record)
    elif record['status'] == 'On Hold':
        record['status'] = 'On hold - قضية عالقة'
        oh = oh+1
        print('oh',oh)
        recordUpdated = fulcrum.records.update(record['id'], record)
    else:
        print('do nothing')

    # recordUpdated = fulcrum.records.update(record['id'], record)

do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing